In [2]:
from path import path
srcdir = path('basic').abspath()
for wrapper in srcdir.walkfiles('wrapper_*.cpp'):
    wrapper.unlink()
wrapper = srcdir/'_module.cpp'
if wrapper.exists():
    wrapper.unlink()
wrapper = srcdir/'_module.py'
if wrapper.exists():
    wrapper.unlink()

In [3]:
import autowig

ImportError: No module named parse

In [ ]:
asg = autowig.AbstractSemanticGraph()

asg = autowig.parser(asg, [self.srcdir/'overload.h', self.srcdir/'binomial.h'],
                          ['-x', 'c++', '-std=c++11', '-I' + str(self.srcdir)],
                          silent = True)

In [ ]:
autowig.controller.plugin = 'default'
autowig.controller(asg)

In [ ]:
wrappers = autowig.generator(asg, module = self.srcdir/'_module.cpp',
                                  decorator = self.srcdir/'_module.py',
                                  prefix = 'wrapper_')

In [ ]:
for wrapper in wrappers:
    wrapper.write()

In [ ]:
autowig.scons(self.srcdir, 'build')

In [ ]:
for wrapper in srcdir.walkfiles('wrapper_*.cpp'):
    wrapper.unlink()
wrapper = srcdir/'_module.cpp'
if wrapper.exists():
    wrapper.unlink()
wrapper = srcdir/'_module.py'
if wrapper.exists():
    wrapper.unlink()